<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Chapter_4_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>